In [52]:
# Bibliotecas
import os
import pandas as pd
import sklearn
import numpy as np 

import datetime

from matplotlib import pyplot as plt
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix

from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support
from sklearn.model_selection import cross_validate
# Models

from sklearn.neural_network import MLPClassifier

from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier


#from sklearn.ensemble import RandomForestRegressor

In [2]:
# Datasets
path1="/Users/fb/Downloads/osp"
os.chdir(path1)
os.getcwd()

'/Users/fb/Downloads/osp'

In [3]:
df_hosp = pd.read_csv('hosp_vfinal.csv', encoding='latin-1')

In [4]:
#df_hosp.columns.tolist()

In [5]:
# New columns para o target
df_hosp['estacao'] = 0
#df_hosp['custo'] = 0
#df_hosp['internacao'] = 0

In [6]:
df_hosp.head()

,Unnamed: 0,SEXO,PROC_REA,VAL_TOT,DT_INTER,IDADE,DIAS_PERM,estado_id,estacao
0,1,1,303140046,479.19,20140119,31,2,12,0
1,2,3,303140046,503.19,20140107,3,3,12,0
2,3,1,303140046,479.19,20140116,19,2,12,0
3,4,3,303140046,495.19,20140117,1,2,12,0
4,5,1,303140046,535.19,20140110,1,7,12,0


In [7]:
# Ajustando data
date = df_hosp.DT_INTER
date = pd.to_datetime(date, format='%Y%m%d', errors='ignore')
df_hosp['ano'] = pd.DatetimeIndex(date).year
df_hosp['mes'] = pd.DatetimeIndex(date).month
df_hosp['dia'] = pd.DatetimeIndex(date).day
#df_hosp['dia']

In [8]:
# Ajustando Data para temperatura/estacao
#df_hosp['estacao'][ df_hosp['mes'] == '1' ] = "1"
### Verao -> 21 de dezembro (19h23) a 20 de março de 2019
df_hosp.loc[(df_hosp['mes'] == 12) & (df_hosp['dia'] >= 21),'estacao'] = 1
df_hosp.loc[(df_hosp['mes'] >= 1) & (df_hosp['mes'] <= 2),'estacao'] = 1
df_hosp.loc[(df_hosp['mes'] == 3) & (df_hosp['dia'] <= 20),'estacao'] = 1
### Outono -> 20 de março (13h15) a 21 de junho (7h07).
df_hosp.loc[(df_hosp['mes'] == 3) & (df_hosp['dia'] >= 20),'estacao'] = 2
df_hosp.loc[(df_hosp['mes'] >= 4) & (df_hosp['mes'] <= 5),'estacao'] = 2
df_hosp.loc[(df_hosp['mes'] == 6) & (df_hosp['dia'] <= 21),'estacao'] = 2
### Inversno -> 21 de junho (7h07) a 22 de setembro
df_hosp.loc[(df_hosp['mes'] == 6) & (df_hosp['dia'] >= 21),'estacao'] = 3
df_hosp.loc[(df_hosp['mes'] >= 7) & (df_hosp['mes'] <= 8),'estacao'] = 3
df_hosp.loc[(df_hosp['mes'] == 9) & (df_hosp['dia'] <= 22),'estacao'] = 3
### Primavera -> 22 de setembro (22h54) a 21 de dezembro 
df_hosp.loc[(df_hosp['mes'] == 9) & (df_hosp['dia'] >= 22),'estacao'] = 4
df_hosp.loc[(df_hosp['mes'] >= 10) & (df_hosp['mes'] <= 11),'estacao'] = 4
df_hosp.loc[(df_hosp['mes'] == 12) & (df_hosp['dia'] <= 21),'estacao'] = 4

In [9]:
#Ajustando sexo
df_hosp['SEXO_2'] = 0
df_hosp.loc[df_hosp['SEXO'] == 3, 'SEXO_2'] = 1
df_hosp = df_hosp.drop('SEXO',1)

In [10]:
### Idade
## 0 - Criança / 1 - Jovem / 2 - Adulto1 / 3 - Adulto2 / 4 - Adulto3 / 5 - Idoso
df_hosp['Idade_index'] = 0
df_hosp.loc[df_hosp['IDADE'] < 12, 'Idade_index'] = 0
df_hosp.loc[(df_hosp['IDADE'] >= 12) & (df_hosp['IDADE'] < 20), 'Idade_index'] = 1
df_hosp.loc[(df_hosp['IDADE'] >= 20) & (df_hosp['IDADE'] < 30), 'Idade_index'] = 2
df_hosp.loc[(df_hosp['IDADE'] >= 30) & (df_hosp['IDADE'] < 40), 'Idade_index'] = 3
df_hosp.loc[(df_hosp['IDADE'] >= 40) & (df_hosp['IDADE'] < 50), 'Idade_index'] = 4
df_hosp.loc[df_hosp['IDADE'] >= 50, 'Idade_index'] = 5
df_hosp = df_hosp.drop('IDADE', 1)
df_hosp.head()

,Unnamed: 0,PROC_REA,VAL_TOT,DT_INTER,DIAS_PERM,estado_id,estacao,ano,mes,dia,SEXO_2,Idade_index
0,1,303140046,479.19,20140119,2,12,1,2014,1,19,0,3
1,2,303140046,503.19,20140107,3,12,1,2014,1,7,1,0
2,3,303140046,479.19,20140116,2,12,1,2014,1,16,0,1
3,4,303140046,495.19,20140117,2,12,1,2014,1,17,1,0
4,5,303140046,535.19,20140110,7,12,1,2014,1,10,0,0


In [11]:
#df_hosp.loc[df_hosp['DIAS_PERM'] < 3, 'DIAS_PERM'] = 0
#df_hosp.loc[df_hosp['DIAS_PERM'] >= 3, 'DIAS_PERM'] = 1

In [12]:
#df_hosp['PROC_REA'] = df_hosp['PROC_REA'].astype('str')
df_hosp['estado_id'] = df_hosp['estado_id'].astype('str')
df_hosp['estacao'] = df_hosp['estacao'].astype('str')
df_hosp['SEXO_2'] = df_hosp['SEXO_2'].astype('str')
df_hosp['Idade_index'] = df_hosp['Idade_index'].astype('str')

In [13]:
df_hosp.loc[df_hosp['VAL_TOT'] < 500, 'custo'] = 0
df_hosp.loc[df_hosp['VAL_TOT'] >= 500, 'custo'] = 1

In [14]:
df_hosp.head()

,Unnamed: 0,PROC_REA,VAL_TOT,DT_INTER,DIAS_PERM,estado_id,estacao,ano,mes,dia,SEXO_2,Idade_index,custo
0,1,303140046,479.19,20140119,2,12,1,2014,1,19,0,3,0.0
1,2,303140046,503.19,20140107,3,12,1,2014,1,7,1,0,1.0
2,3,303140046,479.19,20140116,2,12,1,2014,1,16,0,1,0.0
3,4,303140046,495.19,20140117,2,12,1,2014,1,17,1,0,0.0
4,5,303140046,535.19,20140110,7,12,1,2014,1,10,0,0,1.0


In [15]:
df_hosp = df_hosp.drop([ 'VAL_TOT', 'PROC_REA',
                        'DT_INTER', 'DIAS_PERM',
                        'dia', 'mes'], 1)

In [16]:
df_train = df_hosp[df_hosp['ano'] <= 2016]
df_val = df_hosp[df_hosp['ano'] > 2016]

In [17]:
df_train.head()

,Unnamed: 0,estado_id,estacao,ano,SEXO_2,Idade_index,custo
0,1,12,1,2014,0,3,0.0
1,2,12,1,2014,1,0,1.0
2,3,12,1,2014,0,1,0.0
3,4,12,1,2014,1,0,0.0
4,5,12,1,2014,0,0,1.0


In [18]:
from category_encoders.one_hot import OneHotEncoder
enc = OneHotEncoder(cols=['estado_id',
                          'estacao', 'ano',
                          'SEXO_2', 'Idade_index',
                          'custo'], 
                          use_cat_names=True)
enc.fit(df_train)

OneHotEncoder(cols=['estado_id', 'estacao', 'ano', 'SEXO_2', 'Idade_index', 'custo'],
       drop_invariant=False, handle_missing='value',
       handle_unknown='value', return_df=True, use_cat_names=True,
       verbose=0)

In [19]:
df_train_ohe = enc.transform(df_train)

In [20]:
df_train_ohe = df_train_ohe.astype(str)

In [21]:
df_val_ohe = enc.transform(df_train)

In [22]:
df_val_ohe = df_val_ohe.astype(str)

In [23]:
df_val_ohe = enc.transform(df_val)

In [24]:
df_val_ohe.head()

,Unnamed: 0,estado_id_12,estado_id_13,estado_id_27,estado_id_25,estado_id_15,estado_id_16,estado_id_29,estado_id_31,estado_id_28,...,SEXO_2_0,SEXO_2_1,Idade_index_3,Idade_index_0,Idade_index_1,Idade_index_4,Idade_index_2,Idade_index_5,custo_0.0,custo_1.0
601,1,1,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,1
602,2,1,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,1,0
603,3,1,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,1,0
604,4,1,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,1,0
605,5,1,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,1,0


In [25]:
X_train = df_train_ohe.drop('custo_1.0', axis=1)
X_val = df_train_ohe.drop('custo_1.0', axis=1)

y_train = df_train_ohe['custo_1.0']
y_val = df_train_ohe['custo_1.0']

# Treino dos modelos

## Divisão 70% treino 30% teste
Metodo classico usado por Data Scientists em um primeiro teste

In [26]:
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X_val, y_val, test_size=test_size, random_state=seed)

### Rede Neural

In [27]:
clf0 = MLPClassifier()

In [28]:
clf0.fit(X_train, y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [29]:
nnt = clf0.predict(X_val)

In [30]:
confusion_matrix(y_val, nnt)

array([[185351,      6],
       [     2, 109748]])

In [31]:
accuracy1 = accuracy_score(y_val, nnt)
print("Accuracy Neural Networt: %.2f%%" % (accuracy1 * 100.0))

Accuracy Neural Networt: 100.00%


### Decision tree

In [32]:
clf1 = DecisionTreeClassifier(max_depth=None, min_samples_split=2,
       random_state=0)

In [33]:
clf1.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best')

In [34]:
dt = clf1.predict(X_val)

In [35]:
confusion_matrix(y_val, dt)

array([[185357,      0],
       [     0, 109750]])

In [36]:
accuracy1 = accuracy_score(y_val, dt)
print("Accuracy Decision Tree: %.2f%%" % (accuracy1 * 100.0))

Accuracy Decision Tree: 100.00%


### Random Forest

In [37]:
clf2 = RandomForestClassifier(n_estimators=100, max_depth=2,
                             random_state=0)

In [38]:
clf2.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=2, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [39]:
rf = clf2.predict(X_val)

In [40]:
confusion_matrix(y_val, rf)

array([[185357,      0],
       [ 14205,  95545]])

In [41]:
accuracy1 = accuracy_score(y_val, rf)
print("Accuracy Random Forest: %.2f%%" % (accuracy1 * 100.0))

Accuracy Random Forest: 95.19%


### XGboost

In [42]:
clf3 = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
                                   max_depth=1, random_state=0)

In [43]:
clf3.fit(X_train, y_train) 

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=1.0, loss='deviance', max_depth=1,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              n_iter_no_change=None, presort='auto', random_state=0,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [44]:
xg = clf3.predict(X_val)

In [45]:
confusion_matrix(y_val, xg)

array([[185357,      0],
       [     0, 109750]])

In [46]:
accuracy12 = accuracy_score(y_val, xg)
print("Accuracy Confusion Matrix XGboost: %.2f%%" % (accuracy12 * 100.0))

Accuracy Confusion Matrix XGboost: 100.00%


## Validação Cruzada
Metodo mais refinado e balanceado feito para melhorar o acerto distribuindo em toda população.

In [47]:
clf0 = MLPClassifier()

clf1 = DecisionTreeClassifier(max_depth=None, min_samples_split=2,
       random_state=0)

clf2 = RandomForestClassifier(n_estimators=100, max_depth=2,
                             random_state=0)

clf3 = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
                                   max_depth=1, random_state=0)

In [48]:
cv0_score = cross_val_score(clf0, 
                            X_val, y_val, 
                            cv = 10)

cv1_score = cross_val_score(clf1, 
                            X_val, y_val, 
                            cv = 10)

cv2_score = cross_val_score(clf2, 
                            X_val, y_val, 
                            cv = 10)

cv3_score = cross_val_score(clf3, 
                            X_val, y_val, 
                            cv = 10)

#cv_test_score = cross_validate(clf_teste, df_hosp.drop('DIAS_PERM', 1), df_hosp.DIAS_PERM, cv = 10, scoring = ['explained_variance', 'neg_mean_absolute_error', 'neg_mean_squared_error', 'r2'])

In [49]:
print(cv0_score)

print(cv1_score)

print(cv2_score)

print(cv3_score)

#print(cv_teste_score)

[0.99732303 0.9997628  1.         0.71756294 1.         0.96835078
 0.49574735 0.99335818 0.94056252 0.39149441]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[0.98322659 0.76442682 0.91975873 1.         1.         0.99207075
 0.91847108 0.92846493 0.85970857 0.99190105]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [55]:
print("Accuracy Rede Neural: %.2f%%" % (cv0_score.mean() * 100.0)) 
print("Accuracy Decision Tree: %.2f%%" % (cv1_score.mean() * 100.0)) 
print("Accuracy Random Forest: %.2f%%" % (cv2_score.mean() * 100.0)) 
print("Accuracy XGboost: %.2f%%" % (cv3_score.mean() * 100.0)) 

Accuracy Rede Neural: 85.04%
Accuracy Decision Tree: 100.00%
Accuracy Random Forest: 93.58%
Accuracy XGboost: 100.00%


## ESEMBLE classificação por voto dos modelos JUNTOS
Metodo de UNIAO dos modelos usados quando os mesmos divergem em seus acertos.

In [56]:
eclf1 = VotingClassifier(estimators=[
                          ('nnet', clf0), ('dt', clf1), ('rf', clf2), ('xg', clf3)], voting='hard')

In [57]:
cv_esemble_score = cross_val_score(eclf1, 
                                  X_val, y_val, 
                                  cv = 10)

In [58]:
print(cv_esemble_score)

[0.99749246 0.99959337 1.         1.         1.         0.99207075
 0.96770696 0.99657743 0.949102   1.        ]


In [59]:
print(" ESEMBLE Accuracy: %.2f%%" % (cv_esemble_score.mean() * 100.0)) 


 ESEMBLE Accuracy: 99.03%
